# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [33]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem, bernoulli, beta

import pylab as plt

In [2]:
# datos
df_ori = pd.read_csv('data/cookie_cats.csv')

df = df_ori.copy()

df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
userid,90189.0,4.998412e+06,2.883286e+06,116.0,2512230.0,4995815.0,7496452.0,9999861.0
sum_gamerounds,90189.0,5.187246e+01,1.950509e+02,0.0,5.0,16.0,51.0,49854.0


In [9]:
len(df[df.retention_1 == True]), len(df[df.retention_7 == True])

(40153, 16781)

In [10]:
# transformacion

In [11]:
# Si no ha hecho rondas, no es información relevante porque no ha podido 
# llegar a las puertas ni 30 ni 40

In [12]:
df = df[df.sum_gamerounds != 0]

df.shape, df_ori.shape

((86195, 5), (90189, 5))

In [23]:
# Cogemos el total de jugadores por cada version del jueg
g30_users = df[df.version == 'gate_30'].userid.count()
g40_users = df[df.version == 'gate_40'].userid.count()

g30_users, g40_users

(42763, 43432)

In [24]:
# creamos un dataframe con solo la version y la retención
new_df = df[['version', 'retention_1', 'retention_7']]

new_df.head()

,version,retention_1,retention_7
0,gate_30,False,False
1,gate_30,True,False
2,gate_40,True,False
3,gate_40,False,False
4,gate_40,True,True


In [31]:
# variable para el numero de jugadores retenidos al día o semana siguiente según versión
g30_day_after = new_df[(new_df.version == 'gate_30') & (new_df.retention_1 == True)].version.count()
g40_day_after = new_df[(new_df.version == 'gate_40') & (new_df.retention_1 == True)].version.count()

g30_week_after = new_df[(new_df.version == 'gate_30') & (new_df.retention_7 == True)].version.count()
g40_week_after = new_df[(new_df.version == 'gate_40') & (new_df.retention_7 == True)].version.count()

In [34]:
# testeo para retention_1
inst_ctrl, day_ctrl = g30_users, g30_day_after    # instalados, los que siguieron al día siguiente de la instalación

inst_test, day_test = g40_users, g40_day_after 

In [35]:
# a prioris

a_C, b_C = 1+day_ctrl, 1+inst_ctrl-day_ctrl

a_T, b_T = 1+day_test, 1+inst_test-day_test


beta_control = beta(a_C, b_C)

beta_test = beta(a_T, b_T)

In [36]:
beta_control.mean(), beta_test.mean()

(0.4675318601660236, 0.4621724915964452)

In [37]:
salto = (beta_control.mean() - beta_test.mean())/beta_control.mean()

salto

0.011463108776534007

In [38]:
prob = 1 - beta_test.cdf(0.0024)

prob

1.0

In [5]:
# testeo para retention_7



In [ ]:
# conclusiones
# Explica detalladamente las conclusiones obtenidas de. tu testeo.